In [2]:
!pip install chromadb
!pip install reportlab
!pip install ChromaIndex
!pip install chroma_index

ERROR: Could not find a version that satisfies the requirement ChromaIndex (from versions: none)
ERROR: No matching distribution found for ChromaIndex
ERROR: Could not find a version that satisfies the requirement chroma_index (from versions: none)
ERROR: No matching distribution found for chroma_index


In [3]:
import pandas as pd
import chromadb
from chromadb.utils import embedding_functions
import numpy as np
from sentence_transformers import SentenceTransformer
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import T5ForConditionalGeneration, T5Tokenizer
from transformers import pipeline
from langchain import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyMuPDFLoader
from langchain.chains.question_answering import load_qa_chain
from langchain.document_loaders.pdf import PyPDFDirectoryLoader # Importing PDF loader from Langchain
from langchain.text_splitter import RecursiveCharacterTextSplitter # Importing text splitter from Langchain
from langchain.embeddings import OpenAIEmbeddings # Importing OpenAI embeddings from Langchain
from langchain.schema import Document # Importing Document schema from Langchain
from langchain.vectorstores.chroma import Chroma # Importing Chroma vector store from Langchain
from dotenv import load_dotenv # Importing dotenv to get API key from .env file
from langchain.chat_models import ChatOpenAI # Import OpenAI LLM
from langchain_core.prompts import ChatPromptTemplate
import os # Importing os module for operating system functionalities
import shutil # Importing shutil module for high-level file operations

In [4]:
sheet1 = pd.read_csv(r"D:\Projects(internship)\LLM based Projects\Resume Matcher\candidate sample input dataset - Sheet1.csv")

In [5]:
sheet1.head()

,Name,Contact Details,Location,Job Skills,Experience,Projects,Comments
0,John Doe,john.doe@example.com,New York,Java; Big Data; Hadoop,5 years,Developed a big data processing system using H...,Strong problem-solving skills.
1,Jane Smith,jane.smith@example.com,San Francisco,JavaScript; React; Node.js,3 years,Built a real-time chat application using React...,Excellent in team collaboration.
2,Bob Johnson,bob.johnson@example.com,New York,Java; Spring; Microservices,6 years,Implemented a microservices architecture for a...,Proactive and detail-oriented.
3,Alice Brown,alice.brown@example.com,Chicago,Python; Django; Machine Learning,4 years,Created a machine learning model for predictiv...,Strong analytical skills.
4,Michael Green,michael.green@example.com,Boston,Ruby; Rails; PostgreSQL,7 years,Developed a scalable web application using Rub...,Great leadership qualities.


In [6]:
sheet1 = sheet1.rename(columns={'Contact Details'	: 'Contact Details (email)'})
sheet1.head()

,Name,Contact Details (email),Location,Job Skills,Experience,Projects,Comments
0,John Doe,john.doe@example.com,New York,Java; Big Data; Hadoop,5 years,Developed a big data processing system using H...,Strong problem-solving skills.
1,Jane Smith,jane.smith@example.com,San Francisco,JavaScript; React; Node.js,3 years,Built a real-time chat application using React...,Excellent in team collaboration.
2,Bob Johnson,bob.johnson@example.com,New York,Java; Spring; Microservices,6 years,Implemented a microservices architecture for a...,Proactive and detail-oriented.
3,Alice Brown,alice.brown@example.com,Chicago,Python; Django; Machine Learning,4 years,Created a machine learning model for predictiv...,Strong analytical skills.
4,Michael Green,michael.green@example.com,Boston,Ruby; Rails; PostgreSQL,7 years,Developed a scalable web application using Rub...,Great leadership qualities.


In [7]:
sheet1.shape

(120, 7)

In [8]:
list_of_candidate_details = []
for i in range(120) :
    curr_detail = (str(sheet1.loc[i,'Job Skills'])+'.'+str(sheet1.loc[i,'Projects'])+str(sheet1.loc[i,'Comments'])+"Having an experince of "+str(sheet1.loc[i,'Experience']))
    list_of_candidate_details.append(curr_detail)

In [9]:
list_of_candidate_details

['Java; Big Data; Hadoop.Developed a big data processing system using Hadoop and Spark.Strong problem-solving skills.Having an experince of 5 years',
 'JavaScript; React; Node.js.Built a real-time chat application using React and Node.js.Excellent in team collaboration.Having an experince of 3 years',
 'Java; Spring; Microservices.Implemented a microservices architecture for an e-commerce platform.Proactive and detail-oriented.Having an experince of 6 years',
 'Python; Django; Machine Learning.Created a machine learning model for predictive analytics.Strong analytical skills.Having an experince of 4 years',
 'Ruby; Rails; PostgreSQL.Developed a scalable web application using Ruby on Rails.Great leadership qualities.Having an experince of 7 years',
 'Java; Big Data; Kafka.Designed a big data pipeline using Kafka and Hadoop.Highly motivated and enthusiastic.Having an experince of 5 years',
 'Go; Kubernetes; Docker.Implemented a containerized application using Kubernetes and Docker.Strong

In [10]:
id_of_employee = []
for i in range(len(sheet1)):
    id_of_employee.append(str(i))

In [11]:
id_of_employee

['0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '30',
 '31',
 '32',
 '33',
 '34',
 '35',
 '36',
 '37',
 '38',
 '39',
 '40',
 '41',
 '42',
 '43',
 '44',
 '45',
 '46',
 '47',
 '48',
 '49',
 '50',
 '51',
 '52',
 '53',
 '54',
 '55',
 '56',
 '57',
 '58',
 '59',
 '60',
 '61',
 '62',
 '63',
 '64',
 '65',
 '66',
 '67',
 '68',
 '69',
 '70',
 '71',
 '72',
 '73',
 '74',
 '75',
 '76',
 '77',
 '78',
 '79',
 '80',
 '81',
 '82',
 '83',
 '84',
 '85',
 '86',
 '87',
 '88',
 '89',
 '90',
 '91',
 '92',
 '93',
 '94',
 '95',
 '96',
 '97',
 '98',
 '99',
 '100',
 '101',
 '102',
 '103',
 '104',
 '105',
 '106',
 '107',
 '108',
 '109',
 '110',
 '111',
 '112',
 '113',
 '114',
 '115',
 '116',
 '117',
 '118',
 '119']

In [12]:
# Instantiate chromadb instance. Data is stored on disk (a folder named 'my_vectordb' will be created in the same folder as this file).
chroma_client = chromadb.PersistentClient(path="my_vectordb")

In [13]:
sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name=r"D:\Projects(internship)\LLM based Projects\all-MiniLM-L6-v2")
collection = chroma_client.get_or_create_collection(name="my_collection", embedding_function=sentence_transformer_ef)

In [14]:
for info,id in zip(list_of_candidate_details,id_of_employee):
    collection.upsert(documents= info , ids=[id])

In [15]:
gg = collection.query(query_texts=["Pick up the top 10 profiles for the following job description, We are looking for a skilled UI Developer to join our dynamic team. The ideal candidate will have a strong background in front-end development, with proficiency in HTML, CSS, JavaScript, and modern frameworks like React or Angular. Your primary responsibility will be to create visually appealing and user-friendly web interfaces that enhance user experience and align with our brand guidelines."],
n_results=20)["ids"]

In [16]:
gg

[['77',
  '44',
  '60',
  '112',
  '61',
  '75',
  '96',
  '73',
  '50',
  '9',
  '1',
  '74',
  '119',
  '17',
  '3',
  '59',
  '81',
  '28',
  '64',
  '87']]

In [17]:
print(gg[0])

['77', '44', '60', '112', '61', '75', '96', '73', '50', '9', '1', '74', '119', '17', '3', '59', '81', '28', '64', '87']


In [18]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from langchain import HuggingFacePipeline, PromptTemplate

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained(r"D:\Projects(internship)\LLM based Projects\flan-t5-large")
model = AutoModelForSeq2SeqLM.from_pretrained(r"D:\Projects(internship)\LLM based Projects\flan-t5-large")


pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer, max_new_tokens=15000)

llm = HuggingFacePipeline(
    pipeline=pipe,
    model_kwargs={"temperature": 0.25},
)

In [56]:
def get_conversational_chain():

    prompt_template = """
    You are given a list of candidate details. Each detail contains the candidate's name and other information, separated by periods. Based on the question, return the number of profiles requested. Each profile's name should be on a new line.

    Candidate details format:
    Name. Contact Details. Location. Job Skills. Experience. Projects. Comments.

    Context:
    {context}

    Question:
    {question}

    Answer with the names of the profiles as requested in the question, each on a new line:
    """

    model = llm

    prompt = PromptTemplate(template = prompt_template, input_variables = ["context", "question"])
    chain = load_qa_chain(model , chain_type="stuff", prompt=prompt) #can be any model capable of question answering capabilities and stuff just means take the retrieved context and stuff them into the prompt
    prompt_template = ChatPromptTemplate.from_template(prompt_template)
    return prompt_template

In [57]:
def user_input(user_question):   
    global embeddings
    
    docs = collection.query(query_texts=[user_question]     )["ids"]
    docs = docs[0]
    int_indices = list(map(int, docs))

    row_indices = int_indices

    specific_rows = sheet1.iloc[row_indices]

    def merge_row_to_string(row):
        return '. '.join(row.astype(str))

    merged_strings = specific_rows.apply(merge_row_to_string, axis=1)

    context =  '\n'.join(merged_strings)


    prompt_template = get_conversational_chain()

    prompt = prompt_template.format(context=context, question=user_question)

    model = llm

    response = model.invoke(prompt)


    
    '''response = chain(
        {"input_documents":context, "question": user_question}
        , return_only_outputs=True,)'''

    return context

In [59]:
answer = user_input("Pick up the top 10 profiles for the following job description, We are looking for a skilled UI Developer to join our dynamic team. The ideal candidate will have a strong background in front-end development, with proficiency in HTML, CSS, JavaScript, and modern frameworks like React or Angular. Your primary responsibility will be to create visually appealing and user-friendly web interfaces that enhance user experience and align with our brand guidelines.")
print(answer)

Pam Beesly. pam.beesly@yahoo.com. Los Angeles, CA. JavaScript, Python, Java, React. 3 years. Cloud infrastructure setup. Strong leadership skills
Kyle Reese. kyle.reese@hotmail.com. Dallas, TX. Machine Learning, JavaScript, Flask, Node.js. 2 years. Mobile app. Strong leadership skills
Walter White. walter.white@hotmail.com. Philadelphia, PA. JavaScript, C#, Ruby, AWS. 2 years. Legacy system upgrade. Strong leadership skills
Xander Harris. xander.harris@outlook.com. Philadelphia, PA. Node.js, Microservices, JavaScript, Kubernetes. 6 years. Machine Learning model. Strong leadership skills
Ethan Hunt. ethan.hunt@hotmail.com. San Jose, CA. Node.js, SQL, Django, Kubernetes. 5 years. Microservices architecture. Strong leadership skills
Sam Winchester. sam.winchester@gmail.com. New York, NY. Machine Learning, Big Data, Node.js, Spring Boot. 3 years. Cloud infrastructure setup. Strong leadership skills
Tina Cohen-Chang. tina.cohen-chang@outlook.com. Los Angeles, CA. PostgreSQL, Kubernetes, Jav